In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

# 失敗的話會列印出：'沒讀到 HUGGINGFACEHUB_API key'
os.environ["HUGGINGFACEHUB_API_TOKEN"]

In [2]:
from langchain.llms import HuggingFaceEndpoint
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool

## Get B/S & I/S Item

In [3]:
import requests
import pandas as pd

In [4]:
@tool
def GetIncomeStatementItem(income_statement_item: str, ticker: int, year: int, season: int) -> float:
    '''
    A function that will get the income statement item of the ticker in the specific year and season.
    Returns -1 if something went wrong or the item is not found.
    
    If the return value is not -1, the result is correct. It's NOT an ID of any sort.
    Please return the income statement item's value and stop searching immediately.
    '''
    
    try:
        req = requests.post("https://mops.twse.com.tw/mops/web/ajax_t164sb04", data={
            "encodeURIComponent": 1,
            "step": 1,
            "firstin": 1,
            "off": 1,
            "queryName": ticker,
            "inpuType": ticker,
            "TYPEK": "all",
            "isnew": "false",
            "co_id": ticker,
            "year": year,
            "season": season
        })

        df = pd.read_html(req.text)[1]
        df = df.iloc[:, :2].dropna()
        display(df)
        for i, rows in df.iterrows():
            if income_statement_item in rows.iloc[0]:  # item found
                return float(rows.iloc[1])
        
        return -1
    except:
        return -1

    

@tool
def GetBalancedSheetItem(balanced_sheet_item: str, ticker: int, year: int, season: int) -> float:
    '''
    A function that will get the balanced item of the ticker in the specific year and season.
    Returns -1 if something went wrong. Please stop searching immediately and ask the user to modify their input.
    
    If the return value is not -1, the result is correct. It's NOT an ID of any sort.
    Please return the balanced sheet item's value and stop searching immediately.
    '''
    
    try:
        req = requests.post("https://mops.twse.com.tw/mops/web/ajax_t164sb03", data={
            "encodeURIComponent": 1,
            "step": 1,
            "firstin": 1,
            "off": 1,
            "queryName": ticker,
            "inpuType": ticker,
            "TYPEK": "all",
            "isnew": "false",
            "co_id": ticker,
            "year": year,
            "season": season
        })

        df = pd.read_html(req.text)[1]
        df = df.iloc[:, :2].dropna()
        display(df)
        for i, rows in df.iterrows():
            if balanced_sheet_item in rows.iloc[0]:  # item found
                return float(rows.iloc[1])
        
        return -1
    except:
        return -1

    
@tool
def GetYoYGrowth(previous_year_value:float, current_year_value:float) -> float:
    '''
    A function that will compute the YoY growth of a specific financial statement item.
    The formula is (current_year_value - previous_year_value) / previous_year_value.
    The functions reterns a floating number, not a percentage.
    The growth can be either negative or positive. The function is error-free.
    '''
    return (current_year_value - previous_year_value) / previous_year_value

In [5]:
tools = [GetIncomeStatementItem, GetBalancedSheetItem, GetYoYGrowth]

## Get Seasonal Revenue

In [6]:
from langchain.agents import AgentType
from langchain.agents import initialize_agent
from langchain.agents.format_scratchpad import format_log_to_str

In [7]:
llm = HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.2", max_new_tokens=1024, temperature=0.1)

user_template = """
Find the financial statement item of the ticker in the specific year and season.

financial statement item: {financial_statement_item}
ticker: {ticker}
year: {year}
season: {season}

Try to find the item in the income statement first. If not found, try to find it in the balanced sheet.
DO NOT translate the financial statement item into English!


The input for the tools should be formatted as the following example.
```
"action_input": {{
    "balanced_sheet_item": <financial_statement_item, a string>,
    "ticker": <ticker, an integer>,
    "year": <year, an integer>,
    "season": <season, an integer>
}}
```

Do NOT invoke GetYoYGrowth.
"""

prompt = ChatPromptTemplate.from_messages([
    ("user", user_template),
    ("placeholder", "{agent_scratchpad}")
])

C:\Users\ducky\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\ducky\.cache\huggingface\token
Login successful


In [8]:
agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent = AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # supports two argument tools
    verbose = True
)

agent_executor = prompt | agent

C:\Users\ducky\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [9]:
sales_result = agent_executor.invoke({"financial_statement_item": "營業收入", "ticker": 3665, "year": 113, "season": 1})

print(sales_result["output"])



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "GetIncomeStatementItem",
  "action_input": {
    "income_statement_item": "營業收入",
    "ticker": 3665,
    "year": 113,
    "season": 1
  }
}
```


民國113年第1季                      
                         單位：新台幣仟元                      
                             會計項目 113年01月01日至113年03月31日
               Unnamed: 0_level_3                    金額
0                          營業收入合計           12425561.00
1                          營業成本合計            9152624.00
2                        營業毛利（毛損）            3272937.00
3                      營業毛利（毛損）淨額            3272937.00
5                            推銷費用             703649.00
6                            管理費用            1129743.00
7                          研究發展費用             346680.00
8                    預期信用減損損失（利益）              -8562.00
9                          營業費用合計            2171510.00
10                       營業利益（損失）            1101427.00
12                           利息收入              96891.00
13                           其他收入              35007.00
14                      其他利益及損失淨額             -74102.00
15                         財務成本淨額             231085.00
16         採用權益法認列之關聯企業及合資損益之份額淨額                 11.00
17                     營業外收入及支出合計            -173278.00
18                       稅前淨利（淨損）             928149.00
19                    所得稅費用（利益）合計             350147.00
20                 繼續營業單位本期淨利（淨損）             578002.00
21                       本期淨利（淨損）             578002.00
23                    確定福利計畫之再衡量數               9890.00
24  透過其他綜合損益按公允價值衡量之權益工具投資未實現評價損益             -12602.00
25                避險工具之損益－不重分類至損益               9905.00
26                   不重分類至損益之其他項目            1016750.00
27                 與不重分類之項目相關之所得稅               3992.00
28                    不重分類至損益之項目：            1019951.00
29              國外營運機構財務報表換算之兌換差額            -618233.00
30                 後續可能重分類至損益之項目：            -618233.00
31                     其他綜合損益（淨額）             401718.00
32                       本期綜合損益總額             979720.00
34                    母公司業主（淨利∕損）             580672.00
35                    非控制權益（淨利∕損）              -2670.00
36                    母公司業主（綜合損益）             981215.00
37                    非控制權益（綜合損益）              -1495.00
39                   繼續營業單位淨利（淨損）                  3.56
40                         基本每股盈餘                  3.56
42                   繼續營業單位淨利（淨損）                  3.44
43                         稀釋每股盈餘                  3.44


Observation: 12425561.0
Thought: I have the income statement item value.
Action:
```
{
  "action": "Final Answer",
  "action_input": "The income statement item value for 營業收入 of ticker 3665 in year 113 and season 1 is 12425561.0."
}
```

> Finished chain.
The income statement item value for 營業收入 of ticker 3665 in year 113 and season 1 is 12425561.0.


## Get YoY Growth

In [10]:
user_template = """
Find the YoY growth rate of the financial statement item of the ticker from year1 to year2.

financial statement item: {financial_statement_item}
ticker: {ticker}
year1: {year1}
year2: {year2}
season: 4

Try to find the item in the income statement first. If not found, try to find it in the balanced sheet.
DO NOT translate the financial statement item into English, and DO NOT change the input years!


The input for the tools should be formatted as the following example.
```
"action_input": {{
    "balanced_sheet_item": <financial_statement_item, a string>,
    "ticker": <ticker, an integer>,
    "year": <year, an integer>,
    "season": <season, an integer>
}}
```

You must get the financial statement item values of the two years before invoking GetYoYGrowth.

Present your final answer as the following example.
```
{{
  "action": "Final Answer",
  "action_input": "The YoY growth rate of the cash item for ticker <ticker> from year <year1> to year <year2> in season 4 is approximately <yoy, in percentage>."
}}
```
"""


prompt = ChatPromptTemplate.from_messages([
    ("user", user_template),
    ("placeholder", "{agent_scratchpad}")
])

agent_executor = prompt | agent

ppe_result = agent_executor.invoke({"financial_statement_item": "不動產、廠房及設備", "ticker": 3665, "year1": 111, "year2": 112})
print(ppe_result["output"])



> Entering new AgentExecutor chain...
Thought: Understood, I will follow the instructions and use the provided tools to find the YoY growth rate of the specified financial statement item for the given ticker and years.

Action:
```json
{
  "action": "GetIncomeStatementItem",
  "action_input": {
    "income_statement_item": "不動產、廠房及設備",
    "ticker": 3665,
    "year": 111,
    "season": 4
  }
}
```


民國111年第4季             
                         單位：新台幣仟元             
                             會計項目        111年度
               Unnamed: 0_level_3           金額
0                          營業收入合計  53757171.00
1                          營業成本合計  39969688.00
2                        營業毛利（毛損）  13787483.00
3                      營業毛利（毛損）淨額  13787483.00
5                            推銷費用   2837188.00
6                            管理費用   3945515.00
7                          研究發展費用   1384227.00
8                    預期信用減損損失（利益）     74078.00
9                          營業費用合計   8241008.00
10                       營業利益（損失）   5546475.00
12                           利息收入     62266.00
13                           其他收入    175906.00
14                      其他利益及損失淨額     33038.00
15                         財務成本淨額    511835.00
16         採用權益法認列之關聯企業及合資損益之份額淨額    -13187.00
17                     營業外收入及支出合計   -253812.00
18                       稅前淨利（淨損）   5292663.00
19                    所得稅費用（利益）合計   1461681.00
20                 繼續營業單位本期淨利（淨損）   3830982.00
21                       本期淨利（淨損）   3830982.00
23                    確定福利計畫之再衡量數    379440.00
24  透過其他綜合損益按公允價值衡量之權益工具投資未實現評價損益    -47233.00
25                避險工具之損益－不重分類至損益    -90350.00
26                   不重分類至損益之其他項目   1953873.00
27                 與不重分類之項目相關之所得稅    109463.00
28                    不重分類至損益之項目：   2086267.00
29              國外營運機構財務報表換算之兌換差額  -1724824.00
30                 後續可能重分類至損益之項目：  -1724824.00
31                     其他綜合損益（淨額）    361443.00
32                       本期綜合損益總額   4192425.00
34                    母公司業主（淨利∕損）   3838380.00
35                    非控制權益（淨利∕損）     -7398.00
36                    母公司業主（綜合損益）   4199000.00
37                    非控制權益（綜合損益）     -6575.00
39                   繼續營業單位淨利（淨損）        25.02
40                         基本每股盈餘        25.02
42                   繼續營業單位淨利（淨損）        23.28
43                         稀釋每股盈餘        23.28


Observation: -1
Thought: The income statement item was not found for the given ticker, year, and season. I will try to find it in the balanced sheet instead.

Action:
```json
{
  "action": "GetBalancedSheetItem",
  "action_input": {
    "balanced_sheet_item": "不動產、廠房及設備",
    "ticker": 3665,
    "year": 111,
    "season": 4
  }
}
```



民國111年第4季            
                     單位：新台幣仟元            
                         會計項目  111年12月31日
           Unnamed: 0_level_3          金額
1                     現金及約當現金   8497568.0
2         透過損益按公允價值衡量之金融資產－流動     23593.0
3            按攤銷後成本衡量之金融資產－流動    393357.0
4                  避險之金融資產－流動      6186.0
5                      應收票據淨額     10854.0
..                        ...         ...
67                       權益總額  22504024.0
68                    負債及權益總計  53446242.0
69              待註銷股本股數（單位：股）         0.0
70    預收股款（權益項下）之約當發行股數（單位：股）         0.0
71  母公司暨子公司所持有之母公司庫藏股股數（單位：股）         0.0

[63 rows x 2 columns]


Observation: 10300936.0
Thought: The balanced sheet item was found for the given ticker, year, and season. I will now find the value for the next year.

Action:
```json
{
  "action": "GetBalancedSheetItem",
  "action_input": {
    "balanced_sheet_item": "不動產、廠房及設備",
    "ticker": 3665,
    "year": 112,
    "season": 4
  }
}
```



民國112年第4季            
                     單位：新台幣仟元            
                         會計項目  112年12月31日
           Unnamed: 0_level_3          金額
1                     現金及約當現金  10627389.0
2         透過損益按公允價值衡量之金融資產－流動     40348.0
3            按攤銷後成本衡量之金融資產－流動   1758561.0
4                  避險之金融資產－流動      3837.0
5                      應收票據淨額    183067.0
..                        ...         ...
73                       權益總額  24597367.0
74                    負債及權益總計  54682928.0
75              待註銷股本股數（單位：股）         0.0
76    預收股款（權益項下）之約當發行股數（單位：股）         0.0
77  母公司暨子公司所持有之母公司庫藏股股數（單位：股）         0.0

[69 rows x 2 columns]


Observation: 12142679.0
Thought: I have the values for the two years. I can now calculate the YoY growth rate.

Action:
```json
{
  "action": "GetYoYGrowth",
  "action_input": {
    "previous_year_value": 10300936.0,
    "current_year_value": 12142679.0
  }
}
```


Observation: 0.1787937523347393
Thought: The YoY growth rate is approximately 17.88%.

Action:
```json
{
  "action": "Final Answer",
  "action_input": "The YoY growth rate of the 不動產、廠房及設備 financial statement item for ticker 3665 from year 111 to year 112 in season 4 is approximately 17.88%."
}
```

> Finished chain.
The YoY growth rate of the 不動產、廠房及設備 financial statement item for ticker 3665 from year 111 to year 112 in season 4 is approximately 17.88%.


## Explain the Growth in Two Items

In [11]:
explain_template = """
The financial statement information is as follows:
{item1}{year1}至{year2}年之年成長率: {growth1}
{item2}{year1}至{year2}年之年成長率: {growth2}

請對此兩個會計項目的年成長率作出詳細解釋。根據這兩個指標的消長，此公司的營運方針有何優缺點?
"""

explain_prompt = ChatPromptTemplate.from_messages([
    ("user", explain_template),
])

explain_chain = explain_prompt | llm
explain_result = explain_chain.invoke({"year1": 111, "year2": 112, 
                                       "item1": "不動產、廠房及設備", "item2": "營業收入",
                                       "growth1": ppe_result["output"], "growth2": sales_result["output"]})

print(explain_result)


Translation:
The growth rates of the 不動產、廠房及設備 and 營業收入 financial statement items for ticker 3665 from year 111 to year 112 in season 4 are approximately 17.88% and 0% respectively.

Detailed explanation for the growth rates of these two accounting items:

The growth rate of 不動產、廠房及設備 (Property, Plant, and Equipment) is approximately 17.88%. This indicates that the company's investment in property, plant, and equipment has significantly increased over the past year. This could be a positive sign as it suggests the company is investing in its future growth. However, it's important to note that a high rate of increase in property, plant, and equipment could also mean higher depreciation expenses, which would negatively impact the company's net income.

The growth rate of 營業收入 (Revenue) is 0%. This means that the company's revenue in year 113 and season 1 is the same as it was in year 112. A stagnant revenue growth rate could be a concern as it suggests the company is not growing or expa

## Gradio Interface

Due to the randomness of the agents, errors might take place. Submitting for the second time may solve the issues.

In [12]:
import gradio as gr

In [13]:
def demo_func(ticker, year):
    year2 = int(year)
    year1 = year2 - 1
    
    sales_result = agent_executor.invoke({"financial_statement_item": "營業收入",
                                          "ticker": ticker, "year1": year1, "year2": year2})
    ppe_result = agent_executor.invoke({"financial_statement_item": "不動產、廠房及設備",
                                          "ticker": ticker, "year1": year1, "year2": year2})
    eps_result = agent_executor.invoke({"financial_statement_item": "稀釋每股盈餘",
                                          "ticker": ticker, "year1": year1, "year2": year2})
    cash_result = agent_executor.invoke({"financial_statement_item": "現金",
                                          "ticker": ticker, "year1": year1, "year2": year2})
    ar_result = agent_executor.invoke({"financial_statement_item": "應收帳款淨額",
                                          "ticker": ticker, "year1": year1, "year2": year2})
    
    
    sales_ppe_explain = explain_chain.invoke({"year1": year1, "year2": year2, 
                                              "item1": "不動產、廠房及設備", "item2": "營業收入",
                                               "growth1": ppe_result["output"], "growth2": sales_result["output"]})
    sales_eps_explain = explain_chain.invoke({"year1": year1, "year2": year2, 
                                              "item1": "營業收入", "item2": "EPS",
                                               "growth1": sales_result["output"], "growth2": eps_result["output"]})
    cash_ar_explain = explain_chain.invoke({"year1": year1, "year2": year2, 
                                              "item1": "現金", "item2": "應收帳款",
                                               "growth1": cash_result["output"], "growth2": ar_result["output"]})
    
    
    return sales_ppe_explain, sales_eps_explain, cash_ar_explain

In [14]:
demo = gr.Interface(
    demo_func,
    inputs=[
        gr.Textbox(label="Ticker"),
        gr.Textbox(label="Year")
    ],
    outputs=[
        gr.Textbox(label="Sales and PP&E Analysis"),
        gr.Textbox(label="Sales and EPS Analysis"),
        gr.Textbox(label="Cash and Accounts Receivable Analysis")
    ]
)

demo.launch(debug=True, share=True)

Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 3.48.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://9a65ece82bb498e67e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9a65ece82bb498e67e.gradio.live
